# 01_claim_data_cleaning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime

In [ ]:
path = "../data/raw"
claim_df = pd.read_json(f"{path}/Claim.ndjson", lines=True)
claim_response_df = pd.read_json(f"{path}/ClaimResponse.ndjson", lines=True)
